## DC House Price

#### Project goal: 

- Predict the median house price in DC
- use the right housing statistics and macro variables: (1) time series, (2) acf/pacf plots, (3) stationary test and (4) autocorrelation test
- Modeling: time series linear regression: feature selection in R-squared, AIC, etc.
- Cross validation: Out-of-time performance
- Residual analysis: (1) normality test, (2) Q-Q plot, (3) MAE, (4) MAPE, (5) Homoscedastic test, (6) stationary test and (7) autocorrelation test 

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# tests for stationary 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

import statsmodels.api as sm

#tests for autocorrelation
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox

#test of Heteroscedasticity
from statsmodels.stats.diagnostic import het_breuschpagan

#test for cointegration
from statsmodels.tsa.vector_ar.vecm import coint_johansen 

In [4]:
df = pd.read_csv("../data/full_data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 24 columns):
Time                        118 non-null object
Median Sale Price           118 non-null float64
Median Sale Price MoM       118 non-null float64
Homes Sold                  118 non-null float64
Homes Sold MoM              118 non-null float64
New Listings                118 non-null float64
New Listings MoM            118 non-null float64
Inventory                   118 non-null float64
Inventory MoM               118 non-null float64
Days on Market              118 non-null int64
Days on Market MoM          118 non-null float64
Average Sale To List%       118 non-null float64
Average Sale To List MoM    118 non-null float64
DATE                        117 non-null object
WDXRSA                      117 non-null float64
US_UR                       117 non-null float64
DMV_UR                      117 non-null float64
DC_UR                       117 non-null float64
FED_EM           

In [ ]:
#ACF plot

sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40)
sm.graphics.tsa.plot_pacf(dta.values.squeeze(), lags=40)